In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Import necessary libraries

In [2]:
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from scipy import ndimage
from skimage.measure import regionprops
from skimage import io
from skimage.filters import threshold_otsu   # For finding the threshold for grayscale to binary conversion
import tensorflow as tf
import pandas as pd
import numpy as np
from time import time
import keras
from scipy import ndimage
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg
import numpy as np
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu

##Import Datasets for real and forged images

In [3]:
genuine_image_paths = "/content/drive/MyDrive/signature_identification/real"
forged_image_paths =  "/content/drive/MyDrive/signature_identification/forged"

## Preprocessing the image

In [4]:
def rgb_to_grey(img):
    # Convert RGB image to grayscale
    grey_img = np.mean(img, axis=2)
    return grey_img


In [5]:
# The function grey_to_binary takes a grayscale image (img) as input.
# It applies a Gaussian blur to the image using ndimage.gaussian_filter to remove small components or noise.
# It calculates the threshold value using threshold_otsu.
# The grayscale image is then binarized using this threshold value, and the binary image is returned.

def grey_to_binary(img):
    # Convert grayscale image to binary
    blur_radius = 0.8
    blurred_img = ndimage.gaussian_filter(img, blur_radius)  # Remove small components or noise
    threshold_value = threshold_otsu(blurred_img)
    binary_img = blurred_img > threshold_value
    binary_img = np.logical_not(binary_img)
    return binary_img


In [6]:
# Combined the RGB to grayscale conversion and display into one step using rgb2gray from skimage.color.
# Calculated the threshold value using threshold_otsu from skimage.filters.
# Simplified the display of images using matplotlib.pyplot.imshow and matplotlib.pyplot.show.
# Used more descriptive variable names like grey_img, binary_img, and sign_img for better readability.


def preprocess_image(path, img=None, display=True):
    if img is None:
        img = mpimg.imread(path)

    if display:
        plt.imshow(img)
        plt.show()

    grey_img = rgb2gray(img)  # Convert RGB to grayscale
    if display:
        plt.imshow(grey_img, cmap=cm.Greys_r)
        plt.show()

    threshold_value = threshold_otsu(grey_img)  # Calculate threshold value
    binary_img = grey_img > threshold_value  # Convert grayscale to binary
    if display:
        plt.imshow(binary_img, cmap=cm.Greys_r)
        plt.show()

    r, c = np.where(binary_img == 1)  # Find rows and columns with white pixels (value 1)
    # Create a bounding box around the signature
    sign_img = binary_img[r.min():r.max(), c.min():c.max()]
    if display:
        plt.imshow(sign_img, cmap=cm.Greys_r)
        plt.show()

    return sign_img


## Feature Extraction

In [7]:
# We use np.count_nonzero(img) to count the number of True pixels (white) in the binary image, which is more efficient than looping through each pixel.
# The total number of pixels in the image is calculated using img.shape[0] * img.shape[1].
# The ratio of white pixels to total pixels (ratio) is then calculated and returned.

def calculate_ratio(img):
    # Count the number of True pixels (white) in the binary image
    num_white_pixels = np.count_nonzero(img)

    # Calculate the total number of pixels in the image
    total_pixels = img.shape[0] * img.shape[1]

    # Calculate and return the ratio of white pixels to total pixels
    ratio = num_white_pixels / total_pixels
    return ratio



In [8]:
# We use np.argwhere(img == 1) to find the coordinates of white pixels more efficiently.
# The number of white pixels is determined by the length of the white_pixels array.
# The centroid is calculated using np.mean(white_pixels, axis=0) / img.shape to get the centroid coordinates normalized by the image dimensions.
# The function returns the x and y coordinates of the centroid.

import numpy as np

def calculate_centroid(img):
    # Count the number of white pixels and calculate the centroid
    white_pixels = np.argwhere(img == 1)
    num_white_pixels = len(white_pixels)
    centroid = np.mean(white_pixels, axis=0) / img.shape

    return centroid[0], centroid[1]


In [9]:
# We import regionprops from skimage.measure to calculate region properties of the binary image.
# The function calculate_eccentricity_solidity takes a binary image (img) as input.
# It calculates the region properties using regionprops, assuming there is only one region in the image (regions[0]).
# The function then returns the eccentricity and solidity of the region.

from skimage.measure import regionprops

def calculate_eccentricity_solidity(img):
    # Calculate region properties for the binary image
    regions = regionprops(img.astype("int8"))

    # Return the eccentricity and solidity of the first region (assuming only one region in the image)
    return regions[0].eccentricity, regions[0].solidity


In [10]:
# Calculated the centroid, standard deviation, skewness, and kurtosis using NumPy operations directly on arrays, improving efficiency and readability.

def calculate_skewness_kurtosis(img):
    h, w = img.shape
    x = np.arange(w)  # cols value
    y = np.arange(h)  # rows value

    # Calculate projections along the x and y axes
    xp = np.sum(img, axis=0)
    yp = np.sum(img, axis=1)

    # Calculate centroid
    cx = np.sum(x * xp) / np.sum(xp)
    cy = np.sum(y * yp) / np.sum(yp)

    # Calculate standard deviation
    x2 = (x - cx) ** 2
    y2 = (y - cy) ** 2
    sx = np.sqrt(np.sum(x2 * xp) / np.sum(img))
    sy = np.sqrt(np.sum(y2 * yp) / np.sum(img))

    # Calculate skewness
    x3 = (x - cx) ** 3
    y3 = (y - cy) ** 3
    skewx = np.sum(xp * x3) / (np.sum(img) * sx ** 3)
    skewy = np.sum(yp * y3) / (np.sum(img) * sy ** 3)

    # Calculate Kurtosis
    x4 = (x - cx) ** 4
    y4 = (y - cy) ** 4
    kurtx = np.sum(xp * x4) / (np.sum(img) * sx ** 4) - 3
    kurty = np.sum(yp * y4) / (np.sum(img) * sy ** 4) - 3

    return (skewx, skewy), (kurtx, kurty)


In [11]:
def getFeatures(path, img=None, display=False):
    if img is None:
        img = preprocess_image(path, display=display)
    ratio = calculate_ratio(img)
    centroid = calculate_centroid(img)
    eccentricity, solidity = calculate_eccentricity_solidity(img)
    skewness, kurtosis = calculate_skewness_kurtosis(img)
    return ratio, centroid, eccentricity, solidity, skewness, kurtosis

In [12]:
def getCSVFeatures(path, img=None, display=False):
    if img is None:
        img = preprocess_image(path, display=display)
    temp = getFeatures(path, display=display)
    features = (
        temp[0],
        temp[1][0],
        temp[1][1],
        temp[2],
        temp[3],
        temp[4][0],
        temp[4][1],
        temp[5][0],
        temp[5][1]
    )
    return features

## Saving the features

In [21]:
import os

def makeCSV(genuine_image_paths, forged_image_paths):
    features_folder = '/content/drive/MyDrive/signature_identification/Features'
    training_folder = os.path.join(features_folder, 'Training')
    testing_folder = os.path.join(features_folder, 'Testing')

    folders_to_create = [features_folder, training_folder, testing_folder]
    for folder in folders_to_create:
        if not os.path.exists(folder):
            os.mkdir(folder)
            print(f'New folder "{folder}" created')

    for person in range(1, 13):
        per = ('00' + str(person))[-3:]
        print('Saving features for person id-', per)

        training_file_path = os.path.join(training_folder, f'training_{per}.csv')
        testing_file_path = os.path.join(testing_folder, f'testing_{per}.csv')

        with open(training_file_path, 'w') as training_handle, \
             open(testing_file_path, 'w') as testing_handle:
            training_handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            testing_handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')

            # Genuine signatures
            for i in range(3):
                source = os.path.join(genuine_image_paths, f'{per}{per}_00{i}.png')
                features = getCSVFeatures(path=source)
                training_handle.write(','.join(map(str, features)) + ',1\n')

            for i in range(3, 5):
                source = os.path.join(genuine_image_paths, f'{per}{per}_00{i}.png')
                features = getCSVFeatures(path=source)
                testing_handle.write(','.join(map(str, features)) + ',1\n')

            # Forged signatures
            for i in range(3):
                source = os.path.join(forged_image_paths, f'021{per}_00{i}.png')
                features = getCSVFeatures(path=source)
                training_handle.write(','.join(map(str, features)) + ',0\n')

            for i in range(3, 5):
                source = os.path.join(forged_image_paths, f'021{per}_00{i}.png')
                features = getCSVFeatures(path=source)
                testing_handle.write(','.join(map(str, features)) + ',0\n')

genuine_image_paths = '/content/drive/MyDrive/signature_identification/real'
forged_image_paths = '/content/drive/MyDrive/signature_identification/forged'
makeCSV(genuine_image_paths, forged_image_paths)


Saving features for person id- 001
Saving features for person id- 002
Saving features for person id- 003
Saving features for person id- 004
Saving features for person id- 005
Saving features for person id- 006
Saving features for person id- 007
Saving features for person id- 008
Saving features for person id- 009
Saving features for person id- 010
Saving features for person id- 011
Saving features for person id- 012


In [23]:
makeCSV(genuine_image_paths,forged_image_paths)

Saving features for person id- 001
Saving features for person id- 002
Saving features for person id- 003
Saving features for person id- 004
Saving features for person id- 005
Saving features for person id- 006
Saving features for person id- 007
Saving features for person id- 008
Saving features for person id- 009
Saving features for person id- 010
Saving features for person id- 011
Saving features for person id- 012


##Model

In [26]:
def testing(path):
    feature = getCSVFeatures(path)
    if not(os.path.exists('/content/drive/MyDrive/signature_identification/Features/Testing')):
        os.mkdir('/content/drive/MyDrive/signature_identification/Features/Testing')
    with open('/content/drive/MyDrive/signature_identification/Features/Testing/testcsv.csv', 'w') as handle:
        handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y\n')
        handle.write(','.join(map(str, feature))+'\n')

In [27]:
import tensorflow as tf

ops = tf.compat.v1
ops.reset_default_graph()


In [31]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import cv2
from time import time

# Define ops for TensorFlow 1.x compatibility
ops = tf.compat.v1
ops.reset_default_graph()

n_input = 9
train_person_id = input("Enter person's id : ")
test_image_path = input("Enter path of signature image : ")
train_path = '/content/drive/MyDrive/signature_identification/Features/Training/training_' + train_person_id + '.csv'

def readCSV(train_path):
    df = pd.read_csv(train_path)
    train_input = np.array(df.values[:, :n_input])
    train_input = train_input.astype(np.float32, copy=False)
    correct = np.array(df.values[:, n_input:])
    corr_train = keras.utils.to_categorical(correct, 2)
    return train_input, corr_train

# Parameters
learning_rate = 0.001
training_epochs = 1000

# Network Parameters
n_hidden_1 = 7
n_hidden_2 = 10
n_hidden_3 = 30
n_classes = 2

# tf Graph input
X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random.normal([n_input, n_hidden_1], seed=1)),
    'h2': tf.Variable(tf.random.normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random.normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random.normal([n_hidden_1, n_classes], seed=2))
}
biases = {
    'b1': tf.Variable(tf.random.normal([n_hidden_1], seed=3)),
    'b2': tf.Variable(tf.random.normal([n_hidden_2])),
    'b3': tf.Variable(tf.random.normal([n_hidden_3])),
    'out': tf.Variable(tf.random.normal([n_classes], seed=4))
}

# Create model
def multilayer_perceptron(x):
    layer_1 = tf.tanh((tf.matmul(x, weights['h1']) + biases['b1']))
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    out_layer = tf.tanh(tf.matmul(layer_1, weights['out']) + biases['out'])
    return out_layer

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.math.squared_difference(logits, Y))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# For accuracies
pred = tf.nn.softmax(logits)
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
init = tf.compat.v1.global_variables_initializer()

def extract_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (3, 3))  # Resize to match the input size of the model
    image_flat = image.flatten()
    return image_flat

def determine_signature(train_path, test_image_path):
    train_input, corr_train = readCSV(train_path)
    test_image_features = extract_features(test_image_path)

    with tf.compat.v1.Session() as sess:
        sess.run(init)
        for epoch in range(training_epochs):
            _, cost = sess.run([train_op, loss_op], feed_dict={X: train_input, Y: corr_train})
            if cost < 0.0001:
                break
        prediction = pred.eval({X: test_image_features.reshape((1, n_input))})
        if prediction[0][1] > prediction[0][0]:
            return "Genuine Image"
        else:
            return "Forged Image"

result = determine_signature(train_path, test_image_path)
print("Result:", result)


Enter person's id : 006
Enter path of signature image : /content/drive/MyDrive/signature_identification/real/006006_002.png
Result: Genuine Image


In [30]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import cv2
from time import time

# Disable eager execution
tf.compat.v1.disable_eager_execution()

# Define ops for TensorFlow 1.x compatibility
ops = tf.compat.v1
ops.reset_default_graph()

n_input = 9
train_person_id = input("Enter person's id : ")
test_image_path = input("Enter path of signature image : ")
train_path = '/content/drive/MyDrive/signature_identification/Features/Training/training_' + train_person_id + '.csv'

def readCSV(train_path):
    df = pd.read_csv(train_path)
    train_input = np.array(df.values[:, :n_input])
    train_input = train_input.astype(np.float32, copy=False)
    correct = np.array(df.values[:, n_input:])
    corr_train = keras.utils.to_categorical(correct, 2)
    return train_input, corr_train

# Parameters
learning_rate = 0.001
training_epochs = 1000

# Network Parameters
n_hidden_1 = 7
n_hidden_2 = 10
n_hidden_3 = 30
n_classes = 2

# tf Graph input
X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random.normal([n_input, n_hidden_1], seed=1)),
    'h2': tf.Variable(tf.random.normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random.normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random.normal([n_hidden_1, n_classes], seed=2))
}
biases = {
    'b1': tf.Variable(tf.random.normal([n_hidden_1], seed=3)),
    'b2': tf.Variable(tf.random.normal([n_hidden_2])),
    'b3': tf.Variable(tf.random.normal([n_hidden_3])),
    'out': tf.Variable(tf.random.normal([n_classes], seed=4))
}

# Create model
def multilayer_perceptron(x):
    layer_1 = tf.tanh((tf.matmul(x, weights['h1']) + biases['b1']))
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    out_layer = tf.tanh(tf.matmul(layer_1, weights['out']) + biases['out'])
    return out_layer

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.math.squared_difference(logits, Y))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# For accuracies
pred = tf.nn.softmax(logits)
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
init = tf.compat.v1.global_variables_initializer()

def extract_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (3, 3))  # Resize to match the input size of the model
    image_flat = image.flatten()
    return image_flat

def determine_signature(train_path, test_image_path):
    train_input, corr_train = readCSV(train_path)
    test_image_features = extract_features(test_image_path)

    with tf.compat.v1.Session() as sess:
        sess.run(init)
        for epoch in range(training_epochs):
            _, cost = sess.run([train_op, loss_op], feed_dict={X: train_input, Y: corr_train})
            if cost < 0.0001:
                break
        prediction = pred.eval({X: test_image_features.reshape((1, n_input))})
        if prediction[0][1] > prediction[0][0]:
            return "Genuine Image"
        else:
            return "Forged Image"

result = determine_signature(train_path, test_image_path)
print("Result:", result)


Enter person's id : 006
Enter path of signature image : /content/drive/MyDrive/signature_identification/real/006006_002.png
Result: Genuine Image
